<a href="https://colab.research.google.com/github/Sne1993/Cats-Dogs/blob/main/Exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /root/.config/Google/Drive



In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
dataset_path = "/content/drive/My Drive/kagglecatsanddogs/PetImages"
import os
import numpy as np
import keras
from keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow import data as tf_data

In [4]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]

def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

input_shape = (180, 180, 3)
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)


Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Using 4684 files for validation.


In [5]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)

In [ ]:
print('Experiment 1- Training Cats vs Dogs\n')

print('Learining Rate:- 0.0001, Epoch:- 25, Batch Size:- 64')

epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

# Evaluate the model on the validation dataset
loss, accuracy = model.evaluate(val_ds)

print('Evaluating the model for Experiment 1 cats vs dogs')

print("Validation Loss:", loss,'\n')
print("Validation Accuracy:", accuracy,'\n')


Experiment 1- Training Cats vs Dogs

Learining Rate:- 0.0001, Epoch:- 25, Batch Size:- 64
Epoch 1/25
147/147 ━━━━━━━━━━━━━━━━━━━━ 9295s 63s/step - acc: 0.6253 - loss: 0.6367 - val_acc: 0.4957 - val_loss: 0.6979
Epoch 2/25
147/147 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - acc: 0.7176 - loss: 0.5389 - val_acc: 0.4957 - val_loss: 0.7079
Epoch 3/25
147/147 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - acc: 0.7659 - loss: 0.4643 - val_acc: 0.4957 - val_loss: 0.6907
Epoch 4/25
147/147 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - acc: 0.8004 - loss: 0.4105 - val_acc: 0.5096 - val_loss: 0.6966
Epoch 5/25
 93/147 ━━━━━━━━━━━━━━━━━━━━ 1:10 1s/step - acc: 0.8253 - loss: 0.3697